## Overview Analysis
![img.png](img/SystemStartUp.png)
![SystemStartupAnalysis.svg](img/SystemStartupAnalysis.svg)

In [17]:
package 'Power Roll Up'{
    import ScalarValues::Real;
    import ScalarFunctions::sum;
    
    import 'Roll-Up Pattern'::*;
    import Behavior::*;
    import Structure::*;
    import Analysis_package::*;
    
    value type Power specializes Real;
    
    
    package 'Roll-Up Pattern' {
        block PowerConstraint {
            
            exhibit behavior : 'Behavior::PRBehavior';
            
            part subPower : PowerConstraint [*];
            value power : Power = 0.0;
            value operatingPower : Power = 0.0;
            value initPower : Power = 0.0;
            value totalPower : Power = sum(subPower::totalPower) + power;
        }// END PowerConstraint
        
        package Behavior {
            value type Done;
            value type Init;
            value type TurnOff;
            value type TurnOn;

            state def 'PRBehavior' {    
                entry; then Off;

                state Off {
                    assert power == 0;
                }
                transition off_to_initializing
                    first Off
                    accept Init
                    then initializing;
                transition off_to_on
                    first Off
                    accept TurnOn
                    then On;

                state initializing {
                    assert power == PowerConstraint::initPower;
                }
                transition initializing_to_off
                    first initializing
                    accept Done
                    then Off;

                state On {
                    assert power = PowerConstraint::operatingPower;
                }
                transition on_to_initializing
                    first On
                    accept Init
                    then initializing;
                transition on_to_off
                    first On
                    accept TurnOff
                    then Off;
            } // END state def 'PRBehavior'
        } // END  Behavior   
    } // END package 'Roll-Up Pattern'

    package Structure {
        block System {
            part motor3;
            part motor2;
            part motor1;
        }
    }// END Structure
    
    package Analysis_package {
        block SystemWithCBE specializes System, PowerConstraint {
            
            part redefines motor1 :> 'PowerConstraint' subsets subPower{ 
                value redefines operatingPower = 30;
                value redefines initPower = 50;
            }
            part redefines motor2 :> 'PowerConstraint' subsets subPower{
                value redefines operatingPower = 20;
                value redefines initPower = 15;
            }
            part redefines motor3 :> 'PowerConstraint' subsets subPower{
                value redefines operatingPower = 5;
                value redefines initPower = 10;
            }
        }// END block SystemWithCBE

       activity AnalysisInput specializes TimedActivity{
            assert s1.startTime == s2.startTime;
           
           // start snapshot of s1 == start snapshot of s2
           // snapshot: s1::start then s2::start
           // LTL epxression in RFP? Roadmap? supported by language?
           // 1.1.3 Systems Reasoning requirement in RFP
           // make it an action of "driver"
           // nested action is equivalent of classifier behavior
            action s1 : SendInit;
            action s2 : SendTurnOn;
            then action s3 : SendTurnOff;
        }

        activity TimedActivity {
            value startTime : Real;
            value endTime : Real;
            value duration : Real;
        }
        
        activity SendInit specializes TimedActivity{
            first start;
            then send Init() to Analysis::system::motor1; 
            // not in same context associate analysis inpout with system to be analyzed
            // pass system as parameter
            // provide as reference in activity and provide the action to redefine it
            // must resolve on usage level
            // provide motor as reference
            then done;
        }
        activity SendTurnOff specializes TimedActivity{
            first start;
            then send TurnOff() to Analysis::system::motor2;
            then done;
        }
        activity SendTurnOn specializes TimedActivity{
            first start;
            then send TurnOn() to Analysis::system::motor3;
            then done;
        }
        
        block Analysis {
            assert system::totalPower <= max;
            value max : Power = 100.0;

            part system : SystemWithCBE;

            perform 'AnalysisInput';
            //             
            // solve 'self';
            //
        } // END block Analysis
        
        activity AnalysisResult {
                first start;
                then fork parallel; 
                assert s1.startTime==s2.startTime;
                succession parallel then s1;
                succession parallel then s2;

                action s1 : SendInit;
                then join_node;

                action s2 : SendTurnOn;
                then action s3 : SendTurnOff;
                then join_node;

                join join_node;
                then done;
        }// END AnalysisResult
    } // END package Analysis
}// END Power Roll Up

Package Power Roll Up (664fb138-afe9-4a7e-8b4b-b10fee416a35)
